# Trenowanie LSTM

Tutaj notebook po to żeby ogarnąć jak obsłużyć sekwencje do treningu LSTMa do klasyfikacji (padding itp).
Więc z SupernovaDataset brana jest tylko sekwencja 0 i trenowany jest klasyfikator na tym.
Wykorzystuje Lightning

In [1]:
from supernova.dataset import SupernovaDataset

path = "../data/processed/training_set.pkl"
dataset = SupernovaDataset(path)

In [2]:
next(iter(dataset))

{'object_id': 615,
 'label': tensor(12),
 'metadata': tensor([ 3.4905e+02, -6.1944e+01,  3.2080e+02, -5.1754e+01,  1.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,         nan,  1.7000e-02]),
 'sequences': {0: tensor([[ 5.9819e+04,  6.8788e+00,  3.6332e+00,  0.0000e+00],
          [ 5.9820e+04,  3.9365e+01,  3.7756e+00,  1.0000e+00],
          [ 5.9821e+04, -1.0422e+01,  4.1727e+00,  0.0000e+00],
          [ 5.9822e+04, -6.5485e+01,  4.3629e+00,  1.0000e+00],
          [ 5.9823e+04, -1.1335e+02,  4.0691e+00,  1.0000e+00],
          [ 5.9851e+04, -6.8502e+01,  3.3386e+00,  1.0000e+00],
          [ 5.9874e+04, -9.7353e+01,  3.1340e+00,  1.0000e+00],
          [ 5.9875e+04, -9.7524e+01,  2.9631e+00,  1.0000e+00],
          [ 5.9876e+04, -1.0867e+02,  3.4497e+00,  1.0000e+00],
          [ 5.9877e+04, -1.1691e+02,  3.0978e+00,  1.0000e+00],
          [ 5.9878e+04, -1.0277e+02,  3.1358e+00,  1.0000e+00],
          [ 5.9879e+04, -5.2407e+01,  3.2616e+00,  1.0000e+00],
          [ 5

## Trening

Na razie tylko jeden LSTM na sekwencji 0

In [3]:
from torch.utils.data import Dataset
from supernova.dataset import SupernovaDataset


class SupernovaSequence0Dataset(Dataset):
    def __init__(self, dataset_path: str):
        self.base_dataset = SupernovaDataset(dataset_path)

    def __len__(self):
        return len(self.base_dataset)

    def __getitem__(self, idx):
        item = self.base_dataset[idx]

        # Extract only sequence 0
        sequence = item["sequences"][0]
        label = item["label"]

        return {"sequence": sequence, "label": label, "length": len(sequence)}

In [4]:
import torch
from torch.nn.utils.rnn import pad_sequence


def collate_fn(batch):
    sequences = [item["sequence"] for item in batch]
    labels = torch.stack([item["label"] for item in batch])
    lengths = torch.tensor([item["length"] for item in batch])

    # Pad sequences to the same length
    padded_sequences = pad_sequence(sequences, batch_first=True, padding_value=0)

    return {"sequences": padded_sequences, "labels": labels, "lengths": lengths}

In [5]:
import torch.nn as nn


class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super().__init__()

        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=0.2 if num_layers > 1 else 0,
        )
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x, lengths):
        packed = nn.utils.rnn.pack_padded_sequence(
            x, lengths.cpu(), batch_first=True, enforce_sorted=False
        )
        packed_output, (hidden, _) = self.lstm(packed)
        logits = self.fc(hidden[-1])
        return logits

In [6]:
import pytorch_lightning as pl
import torch.nn as nn
from torch.optim import Adam


class LSTMClassifierTraining(pl.LightningModule):
    def __init__(self, model, learning_rate=1e-3):
        super().__init__()
        self.save_hyperparameters(ignore=["model"])

        self.model = model
        self.criterion = nn.CrossEntropyLoss()

    def forward(self, x, lengths):
        return self.model(x, lengths)

    def training_step(self, batch, batch_idx):
        sequences = batch["sequences"]
        labels = batch["labels"]
        lengths = batch["lengths"]

        logits = self(sequences, lengths)
        loss = self.criterion(logits, labels)
        acc = (logits.argmax(dim=1) == labels).float().mean()

        self.log("train_loss", loss, prog_bar=True)
        self.log("train_acc", acc, prog_bar=True)

        return loss

    def validation_step(self, batch, batch_idx):
        sequences = batch["sequences"]
        labels = batch["labels"]
        lengths = batch["lengths"]

        logits = self(sequences, lengths)
        loss = self.criterion(logits, labels)
        acc = (logits.argmax(dim=1) == labels).float().mean()

        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)

        return loss

    def configure_optimizers(self):
        return Adam(self.parameters(), lr=self.hparams.learning_rate)

In [7]:
from torch.utils.data import DataLoader, random_split
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

# Load dataset
dataset = SupernovaSequence0Dataset("../data/processed/training_set.pkl")

# Split into train/val
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create dataloaders
train_loader = DataLoader(
    train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn, num_workers=4
)
val_loader = DataLoader(
    val_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn, num_workers=4
)

# Initialize model
model = LSTMClassifier(
    input_size=4,  # mjd, flux, flux_err, detected
    hidden_size=128,
    num_layers=2,
    num_classes=14,
)

training = LSTMClassifierTraining(model)

# Callbacks
checkpoint_callback = ModelCheckpoint(
    monitor="val_acc",
    dirpath="../models/checkpoints",
    filename="lstm-{epoch:02d}-{val_acc:.2f}",
    save_top_k=1,
    mode="max",
)

early_stop_callback = EarlyStopping(monitor="val_loss", patience=10, mode="min")

# Trainer
trainer = pl.Trainer(
    max_epochs=4,
    callbacks=[checkpoint_callback, early_stop_callback],
    accelerator="auto",
    devices=1,
    log_every_n_steps=1,
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
/home/mgarbowski/repos/fo-projekt/.venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


In [8]:
# Train
trainer.fit(training, train_loader, val_loader)

/home/mgarbowski/repos/fo-projekt/.venv/lib/python3.12/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:881: Checkpoint directory /home/mgarbowski/repos/fo-projekt/models/checkpoints exists and is not empty.

  | Name      | Type             | Params | Mode  | FLOPs
---------------------------------------------------------------
0 | model     | LSTMClassifier   | 202 K  | train | 0    
1 | criterion | CrossEntropyLoss | 0      | train | 0    
---------------------------------------------------------------
202 K     Trainable params
0         Non-trainable params
202 K     Total params
0.810     Total estimated model params size (MB)
4         Modules in train mode
0         Modules in eval mode
0         Total Flops


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=4` reached.
